### Import the Lirbraries and load the Dataset, Code names and models

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
from datetime import datetime
from Bayesian_Optimization import optimize_rfc, optimize_lr
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (accuracy_score,matthews_corrcoef,
                             classification_report, confusion_matrix)
%matplotlib inline

df = pd.read_csv('pre-precessed_dataset.csv')
codes = open('codes.txt','r').readlines()

clf = joblib.load('RF_model.sav')
logmodel = joblib.load('LR_model.sav')

df.head()

,Length ave. (nm),Diameter ave. (nm),BET (m2/g),Purity (%),Zave (batch),PdI (batch),Zave (12.5 ug/ml),PdI (12.5 ug/ml),Zave (200 ug/ml),PdI (200 ug/ml),...,COOH mmol/g,Endotoxins (EU/mg),Diameter min. (nm),Diameter max. (nm),Type_COOH,Type_NH2,Type_OH,Type_PRISTINE,% Total Impurities,Genotoxicity
0,0.000000,0.198214,0.559322,0.934783,0.036390,0.349333,0.020378,0.234910,0.067882,0.306306,...,0.079208,0.34,0.332558,0.206494,0.0,0.0,0.0,1.0,0.134906,0
1,0.093822,0.283929,0.567797,1.000000,0.066958,0.730667,0.132460,0.375204,0.054205,0.570571,...,0.405941,0.42,0.390698,0.298701,0.0,0.0,1.0,0.0,0.075646,0
2,0.039432,0.342857,0.521186,1.000000,0.035861,0.413333,0.014556,0.097879,0.031408,0.357357,...,1.000000,0.50,0.372093,0.394805,1.0,0.0,0.0,0.0,0.048780,0
3,0.048716,0.796429,0.271186,0.923913,0.011777,0.026667,0.000000,0.000000,0.013171,0.000000,...,0.034653,0.48,0.872093,0.775325,0.0,0.0,0.0,1.0,0.219889,0
4,0.156550,0.387500,0.237288,0.934783,0.045124,0.024000,0.425036,0.365416,0.241641,0.822823,...,0.044554,0.52,0.444186,0.419481,0.0,0.0,1.0,0.0,0.190736,1


### Train-Test Split

In [2]:
from Kennard_Stone import kennardstonealgorithm

Using the Kennart-Stone algorithm, we split the dataset into 2 sets, one for training and one for validation

In [3]:
train, test, train_labels, test_labels = kennardstonealgorithm(df,'Genotoxicity',5)

In [4]:
print('Training Features Shape:', train.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (10, 34)
Training Labels Shape: (10,)
Testing Features Shape: (5, 34)
Testing Labels Shape: (5,)


In [5]:
print('The names of the testing samples are:')
print('=====================================')
for i in test.index:
    print(codes[i][:-1])

The names of the testing samples are:
NRCWE- 040
NRCWE- 045
NRCWE- 048
NM-401
NM-402


### RFE for RF

#### 1st iteration

Step 1: Get the features' importance

In [14]:
# Get numerical feature importances and feature names
importances = list(clf.feature_importances_)
feature_list = list(train.columns)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Length ave. (nm)     Importance: 0.18
Variable: Zave (12.5 ug/ml)    Importance: 0.17
Variable: ROS                  Importance: 0.07
Variable: BET (m2/g)           Importance: 0.06
Variable: Zave  (200 ug/ml)    Importance: 0.06
Variable: PdI (batch)          Importance: 0.05
Variable: PdI  (200 ug/ml)     Importance: 0.05
Variable: Diameter ave. (nm)   Importance: 0.04
Variable: Purity (%)           Importance: 0.04
Variable: Endotoxins (EU/mg)   Importance: 0.03
Variable: % Total Impurities   Importance: 0.03
Variable: PdI  (12.5 ug/ml)    Importance: 0.02
Variable: 200 (ug/ml)_via      Importance: 0.02
Variable: 25 (ug/ml)_prolif    Importance: 0.02
Variable: 200 (ug/ml)_prolif   Importance: 0.02
Variable: OH mmol/g            Importance: 0.02
Variable: COOH mmol/g          Importance: 0.02
Variable: Diameter max. (nm)   Importance: 0.02
Variable: Type_PRISTINE        Importance: 0.02
Variable: Zave (batch)         Importance: 0.01
Variable: 50 (ug/ml)_via       Importanc

Step 2: Exclude features with importance less than 0.04

In [7]:
df_rf = df.copy()

for i in range(len(feature_importances)):
    if feature_importances[i][1]<0.04:
        df_rf.drop(feature_importances[i][0], axis=1, inplace=True)

df_rf.head()

,Length ave. (nm),Diameter ave. (nm),BET (m2/g),Purity (%),PdI (batch),Zave (12.5 ug/ml),Zave (200 ug/ml),PdI (200 ug/ml),ROS,Genotoxicity
0,0.000000,0.198214,0.559322,0.934783,0.349333,0.020378,0.067882,0.306306,0.220779,0
1,0.093822,0.283929,0.567797,1.000000,0.730667,0.132460,0.054205,0.570571,0.389610,0
2,0.039432,0.342857,0.521186,1.000000,0.413333,0.014556,0.031408,0.357357,0.103896,0
3,0.048716,0.796429,0.271186,0.923913,0.026667,0.000000,0.013171,0.000000,0.298701,0
4,0.156550,0.387500,0.237288,0.934783,0.024000,0.425036,0.241641,0.822823,0.220779,1


Step 3: Train - Test split

In [8]:
train_rf, test_rf, train_labels_rf, test_labels_rf = kennardstonealgorithm(df_rf,'Genotoxicity',5)

Step 4: Optimize Random Forest

In [9]:
# Get the time that the optimization started
start_time = datetime.now().strftime("%H:%M:%S")

# Boundaries of the hyperparameters
bo_dict={"n_estimators": (10,250), "min_samples_split": (0.001,0.5), "max_features": (0.5, 0.9)}

# Optimization
rf_optimum = optimize_rfc(train_rf,train_labels_rf,test_rf,test_labels_rf,bo_dict,10,5)

# Get the time that the optimization ended
end_time = datetime.now().strftime("%H:%M:%S")

|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  100.0    |  0.6498   |  0.4754   |  185.7    |
|  2        |  100.0    |  0.7395   |  0.07885  |  47.44    |
|  3        |  100.0    |  0.5232   |  0.4332   |  154.3    |
|  4        |  100.0    |  0.7832   |  0.01127  |  242.8    |
|  5        |  100.0    |  0.833    |  0.107    |  53.64    |
|  6        |  80.0     |  0.6342   |  0.253    |  10.0     |
|  7        |  100.0    |  0.5546   |  0.4438   |  162.1    |
|  8        |  100.0    |  0.6108   |  0.4622   |  176.0    |
|  9        |  100.0    |  0.7635   |  0.08227  |  233.9    |
|  10       |  100.0    |  0.5      |  0.001    |  197.9    |
|  11       |  100.0    |  0.6935   |  0.1891   |  214.3    |
|  12       |  100.0    |  0.9      |  0.001    |  139.5    |
|  13       |  100.0    |  0.5      |  0.5      |  124.7    |
|  14       |  100.0    |  0.9      |  0.001    |  108.8    |
|  15   

In [10]:
print('Minutes to execute:', 
      round((datetime.strptime(end_time, '%H:%M:%S') - datetime.strptime(start_time, '%H:%M:%S')).seconds/60,2))

Minutes to execute: 0.42


Step 5: Test model's performance (on the testing set)

In [11]:
# Declare the model
clf = RandomForestClassifier(n_estimators=rf_optimum['params']['n_estimators'], max_features=rf_optimum['params']['max_features'], min_samples_split=rf_optimum['params']['min_samples_split'],  random_state=42)

# Train the model on training data
clf.fit(train_rf, train_labels_rf);

# Use the model's predict method
predictions = clf.predict(test_rf)

# Print the Train accuracy
print("RF's training accuracy:", accuracy_score(test_labels_rf, predictions))

RF's training accuracy: 1.0


In [12]:
# Print the classification report
print(classification_report(test_labels_rf,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [13]:
cm = confusion_matrix(test_labels_rf,predictions)
print('Confusion Matrix:')
print(cm[0][0],'|',cm[0][1])
print('-----')
print(cm[1][0],'|',cm[1][1])

Confusion Matrix:
4 | 0
-----
0 | 1


In [17]:
tn, fp, fn, tp = confusion_matrix(test_labels_rf,predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print('Specificity:',specificity)
print('Sensitivity:',sensitivity)

Specificity: 1.0
Sensitivity: 1.0


In [14]:
# Print the MCC
print('MCC:', matthews_corrcoef(test_labels_rf,predictions))

MCC: 1.0


In [15]:
#Print the Cross-Validation Score
scores = cross_val_score(clf, train_rf, train_labels_rf, cv=4)

print('List of scores:', scores)
print('Mean of Cross Validtation:', scores.mean())

List of scores: [0.25 1.   1.   0.5 ]
Mean of Cross Validtation: 0.6875


#### 2nd iteration

Step 1: Get the features' importance

In [42]:
# Get numerical feature importances and feature names
importances = list(clf.feature_importances_)
feature_list = list(train_rf.columns)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Purity (%)           Importance: 0.202
Variable: Length ave. (nm)     Importance: 0.162
Variable: PdI  (200 ug/ml)     Importance: 0.155
Variable: Zave (12.5 ug/ml)    Importance: 0.113
Variable: Diameter ave. (nm)   Importance: 0.1
Variable: BET (m2/g)           Importance: 0.088
Variable: Zave  (200 ug/ml)    Importance: 0.087
Variable: ROS                  Importance: 0.051
Variable: OH mmol/g            Importance: 0.026


Step 2: Exclude features with importance less than 0.1

In [94]:
for i in range(len(feature_importances)):
    if feature_importances[i][1]<0.1:
        df_rf.drop(feature_importances[i][0], axis=1, inplace=True)

df_rf.head()

,Length ave. (nm),Diameter ave. (nm),Purity (%),Zave (12.5 ug/ml),PdI (200 ug/ml),Genotoxicity
0,0.000000,0.198214,0.934783,0.020378,0.306306,0.0
1,0.093822,0.283929,1.000000,0.132460,0.570571,0.0
2,0.039432,0.342857,1.000000,0.014556,0.357357,0.0
3,0.048716,0.796429,0.923913,0.000000,0.000000,0.0
4,0.156550,0.387500,0.934783,0.425036,0.822823,1.0


Step 3: Train - Test split

In [17]:
train_rf, test_rf, train_labels_rf, test_labels_rf = kennardstonealgorithm(df_rf,'Genotoxicity',5)

Step 4: Optimize Random Forest

In [19]:
# Get the time that the optimization started
start_time = datetime.now().strftime("%H:%M:%S")

# Boundaries of the hyperparameters
bo_dict={"n_estimators": (10,250), "min_samples_split": (0.001,0.5), "max_features": (0.5, 0.9)}

# Optimization
rf_optimum = optimize_rfc(train_rf,train_labels_rf,test_rf,test_labels_rf,bo_dict,10,5)

# Get the time that the optimization ended
end_time = datetime.now().strftime("%H:%M:%S")

|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  100.0    |  0.6498   |  0.4754   |  185.7    |
|  2        |  100.0    |  0.7395   |  0.07885  |  47.44    |
|  3        |  100.0    |  0.5232   |  0.4332   |  154.3    |
|  4        |  100.0    |  0.7832   |  0.01127  |  242.8    |
|  5        |  100.0    |  0.833    |  0.107    |  53.64    |
|  6        |  80.0     |  0.6342   |  0.253    |  10.0     |
|  7        |  100.0    |  0.5546   |  0.4438   |  162.1    |
|  8        |  100.0    |  0.6108   |  0.4622   |  176.0    |
|  9        |  100.0    |  0.7635   |  0.08227  |  233.9    |
|  10       |  100.0    |  0.5      |  0.001    |  197.9    |
|  11       |  100.0    |  0.6935   |  0.1891   |  214.3    |
|  12       |  100.0    |  0.9      |  0.001    |  139.5    |
|  13       |  100.0    |  0.5      |  0.5      |  124.7    |
|  14       |  100.0    |  0.9      |  0.001    |  108.8    |
|  15   

In [20]:
print('Minutes to execute:', 
      round((datetime.strptime(end_time, '%H:%M:%S') - datetime.strptime(start_time, '%H:%M:%S')).seconds/60,2))

Minutes to execute: 0.43


Step 5: Test model's performance (on the testing set)

In [21]:
# Declare the model
clf = RandomForestClassifier(n_estimators=rf_optimum['params']['n_estimators'], max_features=rf_optimum['params']['max_features'], min_samples_split=rf_optimum['params']['min_samples_split'],  random_state=42)

# Train the model on training data
clf.fit(train_rf, train_labels_rf);

# Use the model's predict method
predictions = clf.predict(test_rf)

# Print the Train accuracy
print("RF's training accuracy:", accuracy_score(test_labels_rf, predictions))

RF's training accuracy: 1.0


In [22]:
# Print the classification report
print(classification_report(test_labels_rf,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [23]:
cm = confusion_matrix(test_labels_rf,predictions)
print('Confusion Matrix:')
print(cm[0][0],'|',cm[0][1])
print('-----')
print(cm[1][0],'|',cm[1][1])

Confusion Matrix:
4 | 0
-----
0 | 1


In [18]:
tn, fp, fn, tp = confusion_matrix(test_labels_rf,predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print('Specificity:',specificity)
print('Sensitivity:',sensitivity)

Specificity: 1.0
Sensitivity: 1.0


In [24]:
# Print the MCC
print('MCC:', matthews_corrcoef(test_labels_rf,predictions))

MCC: 1.0


In [26]:
#Print the Cross-Validation Score
scores = cross_val_score(clf, train_rf, train_labels_rf, cv=4)

print('List of scores:', scores)
print('Mean of Cross Validtation:', scores.mean())

List of scores: [0.25 1.   1.   0.5 ]
Mean of Cross Validtation: 0.6875


#### 3rd iteration

Step 1: Get the features' importance

In [54]:
# Get numerical feature importances and feature names
importances = list(clf.feature_importances_)
feature_list = list(train_rf.columns)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Purity (%)           Importance: 0.357
Variable: Length ave. (nm)     Importance: 0.219
Variable: Zave (12.5 ug/ml)    Importance: 0.188
Variable: Diameter ave. (nm)   Importance: 0.119
Variable: PdI  (200 ug/ml)     Importance: 0.101


Step 2: Exclude features with importance less than 0.1

In [154]:
for i in range(len(feature_importances)):
    if feature_importances[i][1]<0.119:
        df_rf.drop(feature_importances[i][0], axis=1, inplace=True)

df_rf.head()

,Length ave. (nm),Diameter ave. (nm),Purity (%),Zave (12.5 ug/ml),Genotoxicity
0,0.000000,0.198214,0.934783,0.020378,0.0
1,0.093822,0.283929,1.000000,0.132460,0.0
2,0.039432,0.342857,1.000000,0.014556,0.0
3,0.048716,0.796429,0.923913,0.000000,0.0
4,0.156550,0.387500,0.934783,0.425036,1.0


Step 3: Train - Test split

In [29]:
train_rf, test_rf, train_labels_rf, test_labels_rf = kennardstonealgorithm(df_rf,'Genotoxicity',5)

Step 4: Optimize Random Forest

In [66]:
# Get the time that the optimization started
start_time = datetime.now().strftime("%H:%M:%S")

# Boundaries of the hyperparameters
bo_dict={"n_estimators": (10,250), "min_samples_split": (0.001,0.5), "max_features": (0.5, 0.9)}

# Optimization
rf_optimum = optimize_rfc(train_rf,train_labels_rf,test_rf,test_labels_rf,bo_dict,10,5)

# Get the time that the optimization ended
end_time = datetime.now().strftime("%H:%M:%S")

|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  80.0     |  0.6498   |  0.4754   |  185.7    |
|  2        |  80.0     |  0.7395   |  0.07885  |  47.44    |
|  3        |  80.0     |  0.5232   |  0.4332   |  154.3    |
|  4        |  80.0     |  0.7832   |  0.01127  |  242.8    |
|  5        |  80.0     |  0.833    |  0.107    |  53.64    |
|  6        |  60.0     |  0.6342   |  0.253    |  10.0     |
|  7        |  80.0     |  0.5546   |  0.4438   |  162.1    |
|  8        |  80.0     |  0.6108   |  0.4622   |  176.0    |
|  9        |  80.0     |  0.7635   |  0.08227  |  233.9    |
|  10       |  100.0    |  0.5      |  0.001    |  197.9    |


Final result: The optimal model's accuracy is 100.0 and the optimal parameters are n_estimators=197, min_samples_split=0.001 and max_features=0.5


In [31]:
print('Minutes to execute:', 
      round((datetime.strptime(end_time, '%H:%M:%S') - datetime.strptime(start_time, '%H:%M:%S')).seconds/60,2))

Minutes to execute: 0.4


Step 5: Test model's performance (on the testing set)

In [32]:
# Declare the model
clf = RandomForestClassifier(n_estimators=rf_optimum['params']['n_estimators'], max_features=rf_optimum['params']['max_features'], min_samples_split=rf_optimum['params']['min_samples_split'],  random_state=42)

# Train the model on training data
clf.fit(train_rf, train_labels_rf);

# Use the model's predict method
predictions = clf.predict(test_rf)

# Print the Train accuracy
print("RF's training accuracy:", accuracy_score(test_labels_rf, predictions))

RF's training accuracy: 1.0


In [33]:
# Print the classification report
print(classification_report(test_labels_rf,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [34]:
cm = confusion_matrix(test_labels_rf,predictions)
print('Confusion Matrix:')
print(cm[0][0],'|',cm[0][1])
print('-----')
print(cm[1][0],'|',cm[1][1])

Confusion Matrix:
4 | 0
-----
0 | 1


In [19]:
tn, fp, fn, tp = confusion_matrix(test_labels_rf,predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print('Specificity:',specificity)
print('Sensitivity:',sensitivity)

Specificity: 1.0
Sensitivity: 1.0


In [35]:
# Print the MCC
print('MCC:', matthews_corrcoef(test_labels_rf,predictions))

MCC: 1.0


In [36]:
#Print the Cross-Validation Score
scores = cross_val_score(clf, train_rf, train_labels_rf, cv=4)

print('List of scores:', scores)
print('Mean of Cross Validtation:', scores.mean())

List of scores: [0.25 1.   1.   0.5 ]
Mean of Cross Validtation: 0.6875


Hence, the final RF model is the above, using 4 features, and the importance of the features is the following 

In [172]:
# Get numerical feature importances and feature names
importances = list(clf.feature_importances_)
feature_list = list(train_rf.columns)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Length ave. (nm)     Importance: 0.321
Variable: Purity (%)           Importance: 0.3
Variable: Diameter ave. (nm)   Importance: 0.207
Variable: Zave (12.5 ug/ml)    Importance: 0.172


The classification probabilities of the testing samples: 

In [203]:
print('-----------------------------------------------------------------')
print('  Sample                                    Prob(0)    Prob(1)')
print('-----------------------------------------------------------------')
for i in test_rf.index:
    print('{:40} {}'.format(codes[i][:-1],clf.predict_proba(np.array(test_rf.loc[i]).reshape(1, -1))[0]));

-----------------------------------------------------------------
  Sample                                    Prob(0)    Prob(1)
-----------------------------------------------------------------
NRCWE- 040                               [0.78172589 0.21827411]
NRCWE- 041                               [0.68020305 0.31979695]
NRCWE- 042                               [0.87309645 0.12690355]
NRCWE- 045                               [0.19796954 0.80203046]
NRCWE- 048                               [0.50253807 0.49746193]


### RFE for LR

#### 1st iteration

Step 1: Get the features' importance

In [44]:
# Get numerical feature importances and feature names
importances = list(logmodel.coef_.reshape(logmodel.coef_.shape[1]))
feature_list = list(train.columns)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: PdI  (200 ug/ml)     Importance: 6.45
Variable: Length ave. (nm)     Importance: 3.42
Variable: Zave (12.5 ug/ml)    Importance: 3.21
Variable: Peak (ug/ml)         Importance: 1.03
Variable: 200 (ug/ml)_prolif   Importance: 0.79
Variable: Type_OH              Importance: 0.31
Variable: Diameter ave. (nm)   Importance: 0.0
Variable: BET (m2/g)           Importance: 0.0
Variable: Zave (batch)         Importance: 0.0
Variable: PdI  (12.5 ug/ml)    Importance: 0.0
Variable: Zave  (200 ug/ml)    Importance: 0.0
Variable: ROS                  Importance: 0.0
Variable: 0 (ug/ml)_via        Importance: 0.0
Variable: 12.5 (ug/ml)_via     Importance: 0.0
Variable: 25 (ug/ml)_via       Importance: 0.0
Variable: 50 (ug/ml)_via       Importance: 0.0
Variable: 100 (ug/ml)_via      Importance: 0.0
Variable: 200 (ug/ml)_via      Importance: 0.0
Variable: 12.5 (ug/ml)_prolif  Importance: 0.0
Variable: 25 (ug/ml)_prolif    Importance: 0.0
Variable: 50 (ug/ml)_prolif    Importance: 0.0
Variabl

Step 2: Exclude features with zero importance

In [45]:
df_lr = df.copy()

for i in range(len(feature_importances)):
    if feature_importances[i][1]==0.0:
        df_lr.drop(feature_importances[i][0], axis=1, inplace=True)

df_lr.head()

,Length ave. (nm),Purity (%),PdI (batch),Zave (12.5 ug/ml),PdI (200 ug/ml),Peak (ug/ml),200 (ug/ml)_prolif,Type_OH,Genotoxicity
0,0.000000,0.934783,0.349333,0.020378,0.306306,1.0,0.495238,0.0,0
1,0.093822,1.000000,0.730667,0.132460,0.570571,1.0,0.466667,1.0,0
2,0.039432,1.000000,0.413333,0.014556,0.357357,1.0,0.438095,0.0,0
3,0.048716,0.923913,0.026667,0.000000,0.000000,1.0,0.866667,0.0,0
4,0.156550,0.934783,0.024000,0.425036,0.822823,1.0,0.638095,1.0,1


Step 3: Train - Test split

In [46]:
train_lr, test_lr, train_labels_lr, test_labels_lr = kennardstonealgorithm(df_lr,'Genotoxicity',5)

Step 4: Optimize Logistic Regression

In [52]:
# Get the time that the optimization started
start_time = datetime.now().strftime("%H:%M:%S")

# Boundaries of the hyperparameters
bo_dict = {'C': (100,1000)}
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Optimization
lr_optimum = optimize_lr(train_lr,train_labels_lr,test_lr,test_labels_lr,bo_dict,5,3)

# Get the time that the optimization ended
end_time = datetime.now().strftime("%H:%M:%S")

Optimizing for l1 norm
|   iter    |  target   |     C     |
-------------------------------------
|  1        |  80.0     |  437.1    |
|  2        |  80.0     |  955.6    |
|  3        |  80.0     |  758.8    |
|  4        |  80.0     |  100.0    |
|  5        |  80.0     |  100.0    |
|  6        |  80.0     |  999.9    |
|  7        |  80.0     |  100.1    |
|  8        |  80.0     |  1e+03    |
Optimizing for l2 norm
|   iter    |  target   |     C     |
-------------------------------------
|  1        |  80.0     |  437.1    |
|  2        |  80.0     |  955.6    |
|  3        |  80.0     |  758.8    |
|  4        |  80.0     |  100.0    |
|  5        |  80.0     |  100.0    |
|  6        |  80.0     |  999.9    |
|  7        |  80.0     |  100.1    |
|  8        |  80.0     |  1e+03    |


Final result: The optimal model's accuracy is 80.0 and the optimal parameters are C=437.0861069626262 and penalty=l1


In [54]:
print('Minutes to execute:', 
      round((datetime.strptime(end_time, '%H:%M:%S') - datetime.strptime(start_time, '%H:%M:%S')).seconds/60,2))

Minutes to execute: 0.45


Step 5: Test model's performance (on the testing set)

In [56]:
# Declare the model
logmodel = LogisticRegression(C=lr_optimum['params']['C'], penalty=lr_optimum['params']['norm'],random_state=42)

# Train the model on training data
logmodel.fit(train_lr, train_labels_lr);

# Use the model's predict method
predictions = logmodel.predict(test_lr)

# Print the Train accuracy
print("LR's training accuracy:", accuracy_score(test_labels_lr, predictions))

LR's training accuracy: 0.8


In [57]:
# Print the classification report
print(classification_report(test_labels_lr,predictions))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.50      0.67         2

   micro avg       0.80      0.80      0.80         5
   macro avg       0.88      0.75      0.76         5
weighted avg       0.85      0.80      0.78         5



In [58]:
cm = confusion_matrix(test_labels_lr,predictions)
print('Confusion Matrix:')
print(cm[0][0],'|',cm[0][1])
print('-----')
print(cm[1][0],'|',cm[1][1])

Confusion Matrix:
3 | 0
-----
1 | 1


In [30]:
tn, fp, fn, tp = confusion_matrix(test_labels_lr,predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print('Specificity:',specificity)
print('Sensitivity:',sensitivity)

Specificity: 1.0
Sensitivity: 0.5


In [59]:
# Print the MCC
print('MCC:', matthews_corrcoef(test_labels_lr,predictions))

MCC: 0.6123724356957946


In [60]:
#Print the Cross-Validation Score
scores = cross_val_score(logmodel, train_lr, train_labels_lr, cv=4)

print('List of scores:', scores)
print('Mean of Cross Validtation:', scores.mean())

List of scores: [0.66666667 0.66666667 1.         0.5       ]
Mean of Cross Validtation: 0.7083333333333333


#### 2nd iteration

Step 1: Get the features' importance

In [61]:
# Get numerical feature importances and feature names
importances = list(logmodel.coef_.reshape(logmodel.coef_.shape[1]))
feature_list = list(train_lr.columns)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Zave (12.5 ug/ml)    Importance: 17.96
Variable: Type_OH              Importance: 8.58
Variable: 200 (ug/ml)_prolif   Importance: 6.89
Variable: Length ave. (nm)     Importance: 0.0
Variable: PdI  (200 ug/ml)     Importance: 0.0
Variable: Peak (ug/ml)         Importance: 0.0
Variable: PdI (batch)          Importance: -8.58
Variable: Purity (%)           Importance: -15.76


Step 2: Exclude features with zero importance

In [62]:
for i in range(len(feature_importances)):
    if feature_importances[i][1]==0.0:
        df_lr.drop(feature_importances[i][0], axis=1, inplace=True)

df_lr.head()

,Purity (%),PdI (batch),Zave (12.5 ug/ml),200 (ug/ml)_prolif,Type_OH,Genotoxicity
0,0.934783,0.349333,0.020378,0.495238,0.0,0
1,1.000000,0.730667,0.132460,0.466667,1.0,0
2,1.000000,0.413333,0.014556,0.438095,0.0,0
3,0.923913,0.026667,0.000000,0.866667,0.0,0
4,0.934783,0.024000,0.425036,0.638095,1.0,1


Step 3: Train - Test split

In [63]:
train_lr, test_lr, train_labels_lr, test_labels_lr = kennardstonealgorithm(df_lr,'Genotoxicity',5)

Step 4: Optimize Logistic Regression

In [64]:
# Get the time that the optimization started
start_time = datetime.now().strftime("%H:%M:%S")

# Boundaries of the hyperparameters
bo_dict = {'C': (1000,2000)}
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Optimization
lr_optimum = optimize_lr(train_lr,train_labels_lr,test_lr,test_labels_lr,bo_dict,5,3)

# Get the time that the optimization ended
end_time = datetime.now().strftime("%H:%M:%S")

Optimizing for l1 norm
|   iter    |  target   |     C     |
-------------------------------------
|  1        |  100.0    |  1.375e+0 |
|  2        |  100.0    |  1.951e+0 |
|  3        |  100.0    |  1.732e+0 |
|  4        |  100.0    |  1e+03    |
|  5        |  100.0    |  1e+03    |
|  6        |  100.0    |  2e+03    |
|  7        |  100.0    |  1e+03    |
|  8        |  100.0    |  2e+03    |
Optimizing for l2 norm
|   iter    |  target   |     C     |
-------------------------------------
|  1        |  100.0    |  1.375e+0 |
|  2        |  100.0    |  1.951e+0 |
|  3        |  100.0    |  1.732e+0 |
|  4        |  100.0    |  1e+03    |
|  5        |  100.0    |  1e+03    |
|  6        |  100.0    |  2e+03    |
|  7        |  100.0    |  1e+03    |
|  8        |  100.0    |  2e+03    |


Final result: The optimal model's accuracy is 100.0 and the optimal parameters are C=1374.5401188473625 and penalty=l1


In [65]:
print('Minutes to execute:', 
      round((datetime.strptime(end_time, '%H:%M:%S') - datetime.strptime(start_time, '%H:%M:%S')).seconds/60,2))

Minutes to execute: 0.4


Step 5: Test model's performance (on the testing set)

In [66]:
# Declare the model
logmodel = LogisticRegression(C=lr_optimum['params']['C'], penalty=lr_optimum['params']['norm'],random_state=42)

# Train the model on training data
logmodel.fit(train_lr, train_labels_lr);

# Use the model's predict method
predictions = logmodel.predict(test_lr)

# Print the Train accuracy
print("LR's training accuracy:", accuracy_score(test_labels_lr, predictions))

LR's training accuracy: 1.0


In [67]:
# Print the classification report
print(classification_report(test_labels_lr,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [68]:
cm = confusion_matrix(test_labels_lr,predictions)
print('Confusion Matrix:')
print(cm[0][0],'|',cm[0][1])
print('-----')
print(cm[1][0],'|',cm[1][1])

Confusion Matrix:
4 | 0
-----
0 | 1


In [32]:
tn, fp, fn, tp = confusion_matrix(test_labels_lr,predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print('Specificity:',specificity)
print('Sensitivity:',sensitivity)

Specificity: 1.0
Sensitivity: 1.0


In [69]:
# Print the MCC
print('MCC:', matthews_corrcoef(test_labels_lr,predictions))

MCC: 1.0


In [70]:
#Print the Cross-Validation Score
scores = cross_val_score(logmodel, train_lr, train_labels_lr, cv=4)

print('List of scores:', scores)
print('Mean of Cross Validtation:', scores.mean())

List of scores: [0.5 0.5 1.  0.5]
Mean of Cross Validtation: 0.625


#### 3rd iteration

Step 1: Get the features' importance

In [71]:
# Get numerical feature importances and feature names
importances = list(logmodel.coef_.reshape(logmodel.coef_.shape[1]))
feature_list = list(train_lr.columns)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Zave (12.5 ug/ml)    Importance: 15.21
Variable: Type_OH              Importance: 6.97
Variable: 200 (ug/ml)_prolif   Importance: -0.92
Variable: Purity (%)           Importance: -25.65
Variable: PdI (batch)          Importance: -31.59


Step 2: Exclude features with minimum absolute importance

In [72]:
for i in range(len(feature_importances)):
    if feature_importances[i][1]==-0.92:
        df_lr.drop(feature_importances[i][0], axis=1, inplace=True)

df_lr.head()

,Purity (%),PdI (batch),Zave (12.5 ug/ml),Type_OH,Genotoxicity
0,0.934783,0.349333,0.020378,0.0,0
1,1.000000,0.730667,0.132460,1.0,0
2,1.000000,0.413333,0.014556,0.0,0
3,0.923913,0.026667,0.000000,0.0,0
4,0.934783,0.024000,0.425036,1.0,1


Step 3: Train - Test split

In [73]:
train_lr, test_lr, train_labels_lr, test_labels_lr = kennardstonealgorithm(df_lr,'Genotoxicity',5)

Step 4: Optimize Logistic Regression

In [74]:
# Get the time that the optimization started
start_time = datetime.now().strftime("%H:%M:%S")

# Boundaries of the hyperparameters
bo_dict = {'C': (1000,2000)}
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Optimization
lr_optimum = optimize_lr(train_lr,train_labels_lr,test_lr,test_labels_lr,bo_dict,5,3)

# Get the time that the optimization ended
end_time = datetime.now().strftime("%H:%M:%S")

Optimizing for l1 norm
|   iter    |  target   |     C     |
-------------------------------------
|  1        |  100.0    |  1.375e+0 |
|  2        |  100.0    |  1.951e+0 |
|  3        |  100.0    |  1.732e+0 |
|  4        |  100.0    |  1e+03    |
|  5        |  100.0    |  1e+03    |
|  6        |  100.0    |  2e+03    |
|  7        |  100.0    |  1e+03    |
|  8        |  100.0    |  2e+03    |
Optimizing for l2 norm
|   iter    |  target   |     C     |
-------------------------------------
|  1        |  100.0    |  1.375e+0 |
|  2        |  100.0    |  1.951e+0 |
|  3        |  100.0    |  1.732e+0 |
|  4        |  100.0    |  1e+03    |
|  5        |  100.0    |  1e+03    |
|  6        |  100.0    |  2e+03    |
|  7        |  100.0    |  1e+03    |
|  8        |  100.0    |  2e+03    |


Final result: The optimal model's accuracy is 100.0 and the optimal parameters are C=1374.5401188473625 and penalty=l1


In [75]:
print('Minutes to execute:', 
      round((datetime.strptime(end_time, '%H:%M:%S') - datetime.strptime(start_time, '%H:%M:%S')).seconds/60,2))

Minutes to execute: 0.4


Step 5: Test model's performance (on the testing set)

In [76]:
# Declare the model
logmodel = LogisticRegression(C=lr_optimum['params']['C'], penalty=lr_optimum['params']['norm'],random_state=42)

# Train the model on training data
logmodel.fit(train_lr, train_labels_lr);

# Use the model's predict method
predictions = logmodel.predict(test_lr)

# Print the Train accuracy
print("LR's training accuracy:", accuracy_score(test_labels_lr, predictions))

LR's training accuracy: 1.0


In [77]:
# Print the classification report
print(classification_report(test_labels_lr,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [78]:
cm = confusion_matrix(test_labels_lr,predictions)
print('Confusion Matrix:')
print(cm[0][0],'|',cm[0][1])
print('-----')
print(cm[1][0],'|',cm[1][1])

Confusion Matrix:
4 | 0
-----
0 | 1


In [33]:
tn, fp, fn, tp = confusion_matrix(test_labels_lr,predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print('Specificity:',specificity)
print('Sensitivity:',sensitivity)

Specificity: 1.0
Sensitivity: 1.0


In [79]:
# Print the MCC
print('MCC:', matthews_corrcoef(test_labels_lr,predictions))

MCC: 1.0


In [80]:
#Print the Cross-Validation Score
scores = cross_val_score(logmodel, train_lr, train_labels_lr, cv=4)

print('List of scores:', scores)
print('Mean of Cross Validtation:', scores.mean())

List of scores: [0.75 0.5  1.   1.  ]
Mean of Cross Validtation: 0.8125


Hence, the final LR model is the above, using 4 features, and the importance of the features is the following 

In [81]:
# Get numerical feature importances and feature names
importances = list(logmodel.coef_.reshape(logmodel.coef_.shape[1]))
feature_list = list(train_lr.columns)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Zave (12.5 ug/ml)    Importance: 18.96
Variable: Type_OH              Importance: 4.97
Variable: Purity (%)           Importance: -21.24
Variable: PdI (batch)          Importance: -24.59


The classification probabilities of the testing samples: 

In [87]:
print('-----------------------------------------------------------------')
print('  Sample                                    Prob(0)    Prob(1)')
print('-----------------------------------------------------------------')
for i in test_lr.index:
    print('{:40} {}'.format(codes[i][:-1],logmodel.predict_proba(np.array(test_lr.loc[i]).reshape(1, -1))[0]));

-----------------------------------------------------------------
  Sample                                    Prob(0)    Prob(1)
-----------------------------------------------------------------
NRCWE- 040                               [9.99998509e-01 1.49121463e-06]
NRCWE- 042                               [9.99999931e-01 6.92746154e-08]
NRCWE- 047                               [9.99900051e-01 9.99494510e-05]
NM-401                                   [0.75733008 0.24266992]
NM-402                                   [0.01026531 0.98973469]


Hence, the Logistic Regression model (using 4 features) is the best model for the genotoxicity classification of MWCNTs. Below are the metrics on the training dataset:

In [88]:
# Use the model's predict method
predictions = logmodel.predict(train_lr)

# Print the Train accuracy
print("LR's training accuracy:", accuracy_score(train_labels_lr, predictions))

LR's training accuracy: 1.0


In [89]:
# Print the classification report
print(classification_report(train_labels_lr,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5

   micro avg       1.00      1.00      1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [90]:
cm = confusion_matrix(train_labels_lr,predictions)
print('Confusion Matrix:')
print(cm[0][0],'|',cm[0][1])
print('-----')
print(cm[1][0],'|',cm[1][1])

Confusion Matrix:
5 | 0
-----
0 | 5


In [34]:
tn, fp, fn, tp = confusion_matrix(train_labels_lr,predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print('Specificity:',specificity)
print('Sensitivity:',sensitivity)

Specificity: 1.0
Sensitivity: 1.0


In [91]:
# Print the MCC
print('MCC:', matthews_corrcoef(train_labels_lr,predictions))

MCC: 1.0


The classification probabilities of the training samples: 

In [93]:
print('-----------------------------------------------------------------')
print('  Sample                                    Prob(0)    Prob(1)')
print('-----------------------------------------------------------------')
for i in train_lr.index:
    print('{:40} {}'.format(codes[i][:-1],logmodel.predict_proba(np.array(train_lr.loc[i]).reshape(1, -1))[0]));

-----------------------------------------------------------------
  Sample                                    Prob(0)    Prob(1)
-----------------------------------------------------------------
NRCWE- 041                               [9.99999962e-01 3.83361988e-08]
NRCWE- 043                               [0.99645338 0.00354662]
NRCWE- 044                               [7.22976631e-04 9.99277023e-01]
NRCWE- 045                               [0.00463586 0.99536414]
NRCWE- 046                               [0.99851594 0.00148406]
NRCWE- 048                               [9.9999994e-01 6.0059791e-08]
NRCWE- 049                               [1.00000000e+00 3.02581161e-13]
NM-400                                   [3.91782473e-04 9.99608218e-01]
NM-403                                   [2.93295812e-07 9.99999707e-01]
NRCWE-006                                [1.79900945e-05 9.99982010e-01]


### Domain of Applicability

In [94]:
test_names = [codes[i] for i in test_lr.index]

In [95]:
leverage_threshold = 3*train_lr.shape[1]/train_lr.shape[0]
print('The Leverage threshold is:', round(leverage_threshold, 2))

The Leverage threshold is: 1.2


In [96]:
# Labels are the values we want to predict
l_train_labels = np.array(train_labels_lr)
l_test_labels = np.array(test_labels_lr)
# Convert to numpy array
l_test = np.array(test_lr)
l_train = np.array(train_lr)

In [97]:
from numpy.linalg import matrix_power
H=list()
reliability=list()
for i in range(len(l_test)):
    H.append(l_test[i].T@(matrix_power(l_train.T@l_train, -1))@l_test[i])
    if H[i]<=leverage_threshold:
        reliability.append('reliable')
    else:
        reliability.append('unreliable')

LV = [(sample[:-1], round(l_val, 2),rely) for sample, l_val, rely in zip(test_names, H, reliability)]
for i in range(len(l_test)):
    [print('Sample: {:40} Leverage Value: {:5}    Reliability: {:20}'.format(LV[i][0],LV[i][1],LV[i][2]))];

Sample: NRCWE- 040                               Leverage Value:  0.26    Reliability: reliable            
Sample: NRCWE- 042                               Leverage Value:   0.3    Reliability: reliable            
Sample: NRCWE- 047                               Leverage Value:  0.52    Reliability: reliable            
Sample: NM-401                                   Leverage Value:  0.26    Reliability: reliable            
Sample: NM-402                                   Leverage Value:  0.19    Reliability: reliable            
